In [1]:
from djstripe.models import *
from django.conf import settings
from backend.utility import *
from backend.stripe import stripe
from subscription.enum import *

In [2]:
t = SubscriptionAccountType.CONNECT

In [5]:
SubscriptionAccountType.label(t)

'CONNECT'